## Chat Models - Sequential Chain


In [ ]:
import os
%pip install langchain
os.environ['OPENAI_API_KEY'] = 'API_KEY_HERE'

Once you've initiated a call to a language model, it's often useful to make subsequent calls. This method, where the output of one call serves as the input to the next, is especially valuable for more complex tasks.

In this notebook, we'll explore how to implement this using 'sequential chains'. These chains help you link multiple calls together, creating pipelines tailored for specific scenarios.


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain.prompts import PromptTemplate

In [ ]:
# Defining a chat model:
chat = ChatOpenAI(temperature=0.7)

In [ ]:
# Brainstorming Chain:

template = """Act as a social marketer. You are responsible for generating a list of blog post ideas.
Here is some company context to help with your brainstorming: {context}
"""

system_message_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(template=template, input_variables=["context"]),
)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

# Create the chain:
brainstorming_chain = LLMChain(llm=chat, prompt=chat_prompt, output_key="ideas")

In [ ]:
# Outline Chain:
template = """You are a content writer. Your task is to pick an idea from {ideas} and then write a blog post outline.
Here is some company context to help with your outline: {context}
"""
system_message_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(template=template, input_variables=["context", "ideas"]),
)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

outline_chain = LLMChain(llm=chat, prompt=chat_prompt, output_key="outline")

In [ ]:
# Create the chain:
outline_chain = SequentialChain(
    chains=[brainstorming_chain, outline_chain],
    input_variables=["context"],
)

In [ ]:
# Run the chain:
outline_chain.run(
    {
        "context": "Our company is a software company that builds products for developers.",
    }
)

"Title: Exploring Different Software Development Methodologies: Waterfall vs. Agile\n\nOutline:\n\nI. Introduction\n    A. Brief explanation of the significance of software development methodologies\n    B. Introduction to Waterfall and Agile methodologies\n    C. The importance of choosing the right methodology for software development projects\n\nII. Waterfall Methodology\n    A. Definition and overview of the Waterfall methodology\n    B. Sequential nature of the Waterfall methodology\n    C. Key characteristics and stages of the Waterfall methodology\n    D. Advantages and disadvantages of the Waterfall methodology\n    E. Real-world examples of projects where the Waterfall methodology is suitable\n\nIII. Agile Methodology\n    A. Definition and overview of the Agile methodology\n    B. Iterative and incremental nature of the Agile methodology\n    C. Key characteristics and principles of Agile methodology (e.g., Scrum, Kanban)\n    D. Advantages and disadvantages of the Agile meth

## Increasing the chain by a length of 1

You could then increase the sequential chain by adding more chains.


In [ ]:
import langchain

langchain.verbose = True

# Translation chain:
template = "Translate the article outline into {language}: {outline}"
prompt = PromptTemplate.from_template(template)
translation_chain = LLMChain(
    llm=chat, prompt=prompt, output_key="translation", verbose=True
)

# Create the a new SeqwentialChain with the translation chain added:
translated_article_outline_chain = SequentialChain(
    chains=[brainstorming_chain, outline_chain, translation_chain],
    input_variables=["context", "language"],
    verbose=True,
)

In [ ]:
translated_article_outline_chain.run(
    {
        "context": "Our company is a software company that builds products for developers.",
        "language": "French",
    }
)



> Entering new SequentialChain chain...
